In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [5]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)


In [6]:
model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser

In [7]:
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

In [8]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    print(s)

{'answer': '세종대왕은 한글을 창제하고 문화를 발전시키는 등 다양한 업적을 가지고 있습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}
